# SQLAlchemy MetaData

Sqlalchemy中数据库元数据最常见的基础对象称为元数据，表和列。
以下各节将说明如何以面向核心样式以及面向ORM的样式使用这些对象。



## 设置带有表对象的元数据 -- Setting up MetaData with Table objects

当我们使用关系数据库时，我们从中查询的数据库中的基本数据键结构被称为表。在SQLalchemy中，数据库“表”最终由类似命名表的Python对象表示。

为了开始使用SQLalchemy表达语言，我们将希望构造构造的表对象，以代表我们感兴趣的所有数据库表。
该表是通过编程构造的，可以直接使用表构造函数，或通过使用**ORM映射类**间接构造（后来在使用**ORM声明形式**来定义表元数据）。
还可以选择从现有数据库中加载某些或所有表信息，称为**反射**。

无论采用哪种方法，我们始终从一个集合开始，该集合将是我们将表格为元数据对象的表。
该对象本质上是围绕Python Dict 的立面，该词典存储了一系列键入其字符串名称的表对象。
虽然ORM提供了一些有关获取此系列的选项，但我们总是可以选择直接制作一个，看起来像：

In [3]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

from sqlalchemy import MetaData
metadata_obj = MetaData()

In [ ]:
一旦我们有了一个 MetaData 对象，我们就可以声明一些 Table 对象。 
本教程将从经典的 SQLAlchemy 教程模型开始，
该模型有一个名为 user_account 的表，用于存储例如网站的用户，以及一个相关的表 address，用于存储与 user_account 表中的行关联的电子邮件地址。
当根本不使用 ORM 声明模型时，我们直接构造每个 Table 对象，通常将每个对象分配给一个变量，这将是我们在应用程序代码中引用表的方式：

In [4]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

对于上面的示例，当我们希望编写引用数据库中的 user_account 表的代码时，我们将使用 user_table Python 变量来引用它。

> **我什么时候在程序中制作元数据对象？**
> 
> 为整个应用程序拥有一个元数据对象是最常见的情况，在应用程序中的单个位置中以单个位置的模块级变量表示，通常是在“模型”或“ dbschema”类型的软件包中。
> 同样很常见的是，元数据是通过以ORM为中心的注册表或声明性基础类访问的，因此在ORM和核心销售的表对象之间共享相同的元数据。
> 
> 也可以有多个元数据集合； 表对象可以不受限制地引用其他集合中的表对象。 
> 然而，对于彼此相关的 Table 对象组，实际上将它们设置在单个 MetaData 集合中要简单得多，无论是从声明它们的角度还是从 DDL 的角度（即 CREATE 和 DROP) 语句以正确的顺序发出。


### 表的构成

我们可以观察到，Python编写的表构造与SQL Create Table语句有相似之处。
从表名称开始，然后列出每一列，其中每列都有一个名称和数据类型。我们上面使用的对象是：

- Table: 代表数据库表，并将其分配给元数据集合。
- Column: 表示数据库表中的列，并将其分配给表对象。该列通常包括一个字符串名称和一个类型对象。通常通过位于表的关联数组访问列对象的列对象的收集。
- Integer, String: 这些类代表 SQL 数据类型，并且可以传递给 Column，有或没有必要被实例化。  上面，我们想给“name”列一个长度“30”，所以我们实例化了String(30)。 但是对于“id”和“fullname”我们没有指定这些，所以我们可以发送类本身。



In [6]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [7]:
user_table.c.keys()

['id', 'name', 'fullname']

### 声明简单的约束 -- Declaring Simple Constraints

Column("name", Type, 约束...)

Column的属性：
- primary_key
- nullable

> 在 Column 定义中使用 ForeignKey 对象时，我们可以省略该 Column 的数据类型； 它是从相关列自动推断出来的，在上面的例子中是 user_account.id 列的 Integer 数据类型。


In [8]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

### 向数据库发送 DDL 

下面包含表构建和发送DDL的完整代码。

我们已经构建了一个对象结构，表示数据库中的两个数据库表，从根 MetaData 对象开始，然后是两个 Table 对象，每个对象都持有 Column 和 Constraint 对象的集合。 这个对象结构将成为我们对 Core 和 ORM 进行的大多数操作的核心。

我们可以使用此结构可以做的第一件有用的事情是将创建表语句或DDL发射到我们的sqlite数据库中，以便我们可以从它们插入和查询数据。通过在我们的元数据上调用元数据。

In [15]:
from sqlalchemy import ForeignKey
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import MetaData

metadata_obj = MetaData()

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

metadata_obj.create_all(engine)

2023-05-31 09:40:00,459 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-31 09:40:00,461 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-05-31 09:40:00,462 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-31 09:40:00,463 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2023-05-31 09:40:00,464 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-31 09:40:00,466 INFO sqlalchemy.engine.Engine COMMIT


## Using ORM Declarative Forms to Define Table Metadata

前面的示例说明了直接使用 Table 对象，它是 SQLAlchemy 在构造 SQL 表达式时最终如何引用数据库表的基础。 如前所述，SQLAlchemy ORM 围绕称为声明表的表声明过程提供了外观。 Declarative Table 过程实现了与我们在上一节中相同的目标，即构建 Table 对象，但在该过程中也为我们提供了其他东西，称为 ORM 映射类，或简称为“映射类”。 映射类是使用 ORM 时最常见的 SQL 基础单元，在现代 SQLAlchemy 中也可以非常有效地用于以核心为中心的使用。

使用声明表的一些好处包括：

- 设置列定义的更简洁，更简洁的曲调风格，其中Python类型可用于表示数据库中要使用的SQL类型

- 由此产生的映射类可用于形成SQL表达式，在许多情况下，PEP 484键入信息，这些信息是由静态分析工具（例如MyPy和IDE类型检查器）拾取的

- 允许声明表格元数据和ORM映射的类，以持久/对象加载操作中使用。

本节将说明使用声明表构建的上一节的同一表元数据。

使用ORM时，我们声明表元数据的过程通常与声明映射类的过程结合使用。映射类是我们要创建的任何Python类，然后将其属性链接到数据库表中的列。尽管有几种如何实现，但最常见的样式被称为声明性，并允许我们一次声明我们的用户定义的类和表格元数据。

### Establishing a Declarative Base

使用 ORM 时，元数据集合仍然存在，但它本身与通常称为声明性基础的 ORM-only 构造相关联。 获取新的 Declarative Base 最方便的方法是创建一个新类，该类是 SQLAlchemy DeclarativeBase 类的子类：


In [12]:
from sqlalchemy.orm import DeclarativeBase
class Base(DeclarativeBase):  # 创建一个基类， 其他表的声明继承这个基类实现
    pass

上面，我们将称之为声明性基础。
当我们制作新的类是基本子类（结合适当的类级指令）的新类时，它们将在类创建时间时将它们建立为新的ORM映射类，通常每个人（但不是独家）指的是特定的表对象。

声明性的基础是指为我们自动创建的元数据集合，假设我们没有从外部提供。该元数据集合可通过DectarativeBase.Metadata类级属性访问。当我们创建新的映射类时，他们每个人都会在此元数据集合中引用一个表：

In [6]:
Base.metadata

MetaData()

In [7]:
Base.registry

## Declaring Mapped Classes

建立基类后，我们现在可以根据新类 User 和 Address 为 user_account 和 address 表定义 ORM 映射类。 我们在下面说明了声明式的最现代形式，它是使用特殊类型 Mapped 从 PEP 484 类型注释驱动的，它指示要映射为特定类型的属性：


In [13]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy import ForeignKey

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    test: Mapped[str]
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

上面的两个类 User 和 Address 现在被称为 ORM 映射类，可用于 ORM 持久化和查询操作，稍后将对此进行描述。这些课程的详情包括:
    
- 每个类是指作为声明映射过程的一部分生成的表对象，该对象是通过将字符串分配给`DeclarativeBase.__tablename__`属性来命名的。创建了类后，该生成的表将从`DectarativeBase.__Table__`属性中获得。

- 如前所述，此表单称为“声明表配置”。有几种可选的声明样式，其中之一就是让我们直接构建 Table 对象，并将其直接分配给 `DeclarativeBase.__table__`这种样式称为带祈使表的声明式。

- 要指示表中的列，我们将使用Mapped_column()构造与基于映射类型的键入注释结合使用。该对象将生成应用于表构建的列对象。

- 对于具有简单数据类型且没有其他选项的列，我们可以单独指示映射类型注释，使用简单的 Python 类型，如 int 和 str 来表示整数和字符串。 在声明性映射过程中如何解释 Python 类型的自定义是非常开放的； 请参阅使用注释声明表（mapped_column() 的类型注释形式）和为背景自定义类型映射部分。

- 使用显式类型注释是完全可选的。 我们也可以使用不带注释的 mapped_column()。 使用这种形式时，我们会根据需要在每个 mapped_column() 构造中使用更明确的类型对象，如 Integer 和 String 以及 nullable=False。

- 两个额外的属性 User.address 和 Address.user 定义了一种不同的relationship()属性 ，它具有类似的注释感知配置样式，如图所示。在使用 ORM 相关对象时会更全面地讨relationship()结构。

- 如果我们不声明自己的类，则会自动为这些类提供一个 `__init__()` 方法。 此方法的默认形式接受所有属性名称作为可选关键字参数：

- 添加`__repr __()`方法，以便我们获得可读的字符串输出；不需要这些方法在这里。与`__init __()`一样，可以使用Dataclasses功能自动生成`__repr __()`方法。


In [14]:
Base.metadata.create_all(engine)

2023-05-31 10:21:48,106 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-31 10:21:48,108 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-05-31 10:21:48,109 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-31 10:21:48,112 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2023-05-31 10:21:48,113 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-31 10:21:48,114 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2023-05-31 10:21:48,116 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-31 10:21:48,117 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2023-05-31 10:21:48,118 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-31 10:21:48,121 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2023-05-31 10:21:48,122 INFO sqlalchemy.engine.Engine [no key 0.00098s] ()
2023-05-31 10:21:48,124 INFO sqlalchemy.engine.

## Table Reflection - 反射

从数据库表加载 Python SQLAlchemy 表对象。


In [17]:
user_account = Table("user_account", metadata_obj, autoload_with=engine)
user_account

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)